In [ ]:
import os
import spacy
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Si cela ne marche pas, vous devez installer le modèle de spacy avec
# la commande suivante dans le powershell anaconda:
# python -m spacy download fr_core_news_lg
nlp = spacy.load("fr_core_news_lg")

# Code pour générer le corpus tokenisé à partir du corpus TXT
files = os.listdir("CORPUS TXT")
for filename in files:
    list_ = open("CORPUS TXT/"+filename, encoding='utf-8').read().split(' ')
    separator = ' '
    txt = separator.join(list_)
    tokenizer = RegexpTokenizer(r'\w+')
    tokenized = tokenizer.tokenize(txt)
    stopWords = set(stopwords.words('french'))
    clean_words = []
    for token in tokenized:
        if token not in stopWords:
            clean_words.append(token)
    with open("CORPUS TOKEN/out_" + filename, "a") as txtfile:
        for word in clean_words:
            print(word, file=txtfile)

In [ ]:
# Fonction de lemmatisation
def return_lemma(sentence):
    doc = nlp(sentence)
    return [str(X.lemma_) for X in doc if X.pos_ in(('NOUN','VERB','ADJ')) and str(X) != '…' and not X.is_punct]

In [ ]:
# Code pour générer le corpus lemmatisé à partir du corpus TXT
files = os.listdir("CORPUS TXT")
for filename in files:
    list_ = open("CORPUS TXT/"+filename, encoding='utf-8').read().split(' ')
    separator = ' '
    txt = separator.join(list_)
    lemma_list = return_lemma(txt)
    with open("CORPUS LEMMA/L_out_" + filename, "a", encoding="utf-8") as txtfile:
        for lemma in lemma_list:
            print(lemma, file=txtfile)

In [ ]:
import os

# Cette partie de code est dediée à calculer la matrice de distances entre
# chaque fichier txt du corpus lemmatisé.
# REMARQUE: Cela peut prendre un peu de temps!
# REMARQUE2: On utilise la similarité cosinus comme distance
files = os.listdir("CORPUS LEMMA")
matrix_finale = []
for filename in files:
    distances = []
    file1 = open("CORPUS LEMMA/"+filename, encoding='utf-8').read().split(' ')
    txt1 = ' '.join(file1)
    doc1 = nlp(txt1)
    for filename2 in files:
        file2 = open("CORPUS LEMMA/"+filename2, encoding='utf-8').read().split(' ')
        txt2 = ' '.join(file2)
        doc2 = nlp(txt2)
        distances.append((1-doc1.similarity(doc2)))
    matrix_finale.append(distances)
matrix_finale

import pickle
# Code pour enregistrer la matrice de distances comme un fichier pkl
# pour ne pas avoir besoin de faire marcher le code d'avant à chaque fois
with open('matrix.pkl', 'wb') as file:
    pickle.dump(matrix_finale, file)

In [ ]:
# Code pour charger la matrice de distances
import pickle
with open('matrix.pkl', 'rb') as file:
    matrix_finale = pickle.load(file)
    print(matrix_finale)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.spatial.distance as ssd
from matplotlib import pyplot as plot
%matplotlib inline
import os

# Liste avec les noms des fichiers lemmatisés
files = os.listdir("CORPUS LEMMA")

# Conversion de la matrice entière en une matrice condensée
distArray = ssd.squareform(matrix_finale)

# On pourrait utiliser aussi le paquet sklearn, mais on a utilisé le paquet scipy car l'affichage avec celui-ci est
# plus facile
Z = linkage(distArray, 'average')
figure = plot.figure(figsize=(50, 30))
dendro = dendrogram(Z, leaf_rotation=0,labels=files, leaf_font_size =12, orientation = 'right')